In [2]:
from tqdm import tqdm_notebook as tqdm

import numpy as np
import pandas as pd

import torch

from matplotlib import pyplot as plt
import seaborn as sns

from transformers import BertTokenizer, BertModel

## e002 の valid を確認する

In [45]:
trn_df = pd.read_csv('../inputs/origin/train.csv')
display(trn_df.shape, trn_df.head())

(27481, 4)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [4]:
ls ../checkpoints/e002/0/

epoch_0_5.76938_-1.00000_0.38606_checkpoint.pth
epoch_1_3.15424_-1.00000_0.67961_checkpoint.pth
epoch_2_2.72038_-1.00000_0.70786_checkpoint.pth
epoch_3_2.68010_-1.00000_0.71049_checkpoint.pth
epoch_4_2.68551_-1.00000_0.71749_checkpoint.pth
epoch_5_2.83366_-1.00000_0.71943_checkpoint.pth
epoch_6_2.97792_-1.00000_0.72161_checkpoint.pth
epoch_7_3.11625_-1.00000_0.72562_checkpoint.pth
epoch_8_3.25741_-1.00000_0.72377_checkpoint.pth
epoch_9_3.34007_-1.00000_0.72597_checkpoint.pth


In [42]:
ckpt = torch.load('../checkpoints/e002/0/epoch_9_3.34007_-1.00000_0.72597_checkpoint.pth')

In [43]:
ckpt.keys()

dict_keys(['fold_num', 'current_epoch', 'model_state_dict', 'optimizer_state_dict', 'scheduler_state_dict', 'val_textIDs', 'val_input_ids', 'val_preds', 'val_labels', 'histories'])

In [46]:
trn_df = trn_df.query(f"textID in {list(ckpt['val_textIDs'])}")
trn_df

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe....,Wow... u just became cooler.,positive
11,16fab9f95b,I really really like the song Love Story by Ta...,like,positive
...,...,...,...,...
27467,432e6de6c9,morning twit-friends! welcome to my new followers,welcome,positive
27469,ac92790d8b,94 more days till BH comes back to LA,94 more days till BH comes back to LA,neutral
27472,8f5adc47ec,http://twitpic.com/663vr - Wanted to visit the...,were too late,negative
27474,8f14bb2715,So I get up early and I feel good about the da...,I feel good ab,positive


In [47]:
def _get_predicted_texts(input_ids, y_preds_head, y_preds_tail, tokenizer):
    predicted_texts = []
    for input_id, y_pred_head, y_pred_tail \
            in zip(input_ids, y_preds_head, y_preds_tail):
        pred_label_head = y_pred_head.argmax()
        pred_label_tail = y_pred_tail.argmax()
        predicted_text = tokenizer.decode(
            input_id[pred_label_head:pred_label_tail + 1])
        predicted_texts.append(predicted_text)

    return predicted_texts

In [55]:
def _get_selected_texts(input_ids, labels_head, labels_tail, tokenizer):
    predicted_texts = []
    for input_id, pred_label_head, pred_label_tail \
            in zip(input_ids, labels_head, labels_tail):
        predicted_text = tokenizer.decode(
            input_id[pred_label_head:pred_label_tail + 1])
        predicted_texts.append(predicted_text)

    return predicted_texts

In [48]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [58]:
predicted_texts = _get_predicted_texts(ckpt['val_input_ids'], ckpt['val_preds'][0],  ckpt['val_preds'][1], tokenizer)
trn_df['predicted_texts'] = predicted_texts
trn_df

,textID,text,selected_text,sentiment,predicted_texts
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"i ` d have responded, if i were going [SEP] [PAD]"
3,9642c003ef,what interview! leave me alone,leave me alone,negative,! leave me alone [SEP] [PAD]
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"sons of * * * *, why couldn"
9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe....,Wow... u just became cooler.,positive,... u just became cooler. hehe
11,16fab9f95b,I really really like the song Love Story by Ta...,like,positive,i really really like the song
...,...,...,...,...,...
27467,432e6de6c9,morning twit-friends! welcome to my new followers,welcome,positive,welcome to my
27469,ac92790d8b,94 more days till BH comes back to LA,94 more days till BH comes back to LA,neutral,94 more days till bh comes back to la [SEP] [PAD]
27472,8f5adc47ec,http://twitpic.com/663vr - Wanted to visit the...,were too late,negative,too late [SEP] [PAD]
27474,8f14bb2715,So I get up early and I feel good about the da...,I feel good ab,positive,good about the


In [59]:
manual_selected_texts = _get_selected_texts(ckpt['val_input_ids'], ckpt['val_labels'][0],  ckpt['val_labels'][1], tokenizer)
trn_df['manual_selected_text'] = manual_selected_texts
trn_df

,textID,text,selected_text,sentiment,predicted_texts,manual_selected_text
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"i ` d have responded, if i were going [SEP] [PAD]","i ` d have responded, if i were going [SEP] [PAD]"
3,9642c003ef,what interview! leave me alone,leave me alone,negative,! leave me alone [SEP] [PAD],leave me alone [SEP] [PAD]
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"sons of * * * *, why couldn","sons of * * * *, why couldn"
9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe....,Wow... u just became cooler.,positive,... u just became cooler. hehe,wow... u just became cooler. hehe
11,16fab9f95b,I really really like the song Love Story by Ta...,like,positive,i really really like the song,like the song
...,...,...,...,...,...,...
27467,432e6de6c9,morning twit-friends! welcome to my new followers,welcome,positive,welcome to my,welcome to my
27469,ac92790d8b,94 more days till BH comes back to LA,94 more days till BH comes back to LA,neutral,94 more days till bh comes back to la [SEP] [PAD],94 more days till bh comes back to la [SEP] [PAD]
27472,8f5adc47ec,http://twitpic.com/663vr - Wanted to visit the...,were too late,negative,too late [SEP] [PAD],were too late [SEP] [PAD]
27474,8f14bb2715,So I get up early and I feel good about the da...,I feel good ab,positive,good about the,and i feel good about the


In [60]:
ckpt['val_labels'][0][5],  ckpt['val_labels'][1][5]

(tensor(11), tensor(13))

In [61]:
tokenizer.encode_plus(trn_df.loc[11]['selected_text'])

{'input_ids': [101, 2066, 102],
 'token_type_ids': [0, 0, 0],
 'attention_mask': [1, 1, 1]}

In [78]:
tokenizer.encode_plus(trn_df.loc[11]['selected_text'], add_special_tokens=False)['input_ids']

[2066]

In [52]:
def jaccard(str1, str2):
    a = set(str1.lower().split())
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [53]:
temp_jac = 0
for i, row in trn_df.iterrows():
    temp_jac += jaccard(row['selected_text'], row['predicted_texts'])

In [54]:
temp_jac / len(trn_df)

0.44581498228808364

## 改善の必要あり
 - decode すると違う形式になってしまう
 - selected_text を encode -> decode して形式が変わるパターンを除去する後処理が必要そう

In [75]:
trn_df.loc[0]

textID                                                         cb774db0d1
text                                  I`d have responded, if I were going
selected_text                         I`d have responded, if I were going
sentiment                                                         neutral
predicted_texts         i ` d have responded, if i were going [SEP] [PAD]
manual_selected_text    i ` d have responded, if i were going [SEP] [PAD]
Name: 0, dtype: object

In [74]:
jaccard(trn_df.loc[0]['selected_text'], trn_df.loc[0]['predicted_texts'][:-12])

0.6666666666666666

In [72]:
trn_df.loc[0]['selected_text'][4:], trn_df.loc[0]['predicted_texts'][6:-12]

('have responded, if I were going', 'have responded, if i were going')